In [ ]:
#import packages
import pandas as pd
import numpy as np
import keras 
import tensorflow as tf
import copy
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score

In [ ]:
#define functions

#reshape data for models other than CNN and VGG
def reshape_data_general(raw_data):
    y = copy.deepcopy(raw_data['emotion'])
    data = raw_data.drop(columns = ['emotion'])
    x = []
    for i in range(data.shape[0]):
        sample = copy.deepcopy(data.loc[i,' pixels'])
        sample = sample.split(' ')
        sample = np.array(list(map(int, sample)))/255-1
        x.append(sample)
    x = np.array(x)
    return x,y
#reshape data for CNN and VGG
def reshape_data_CNN(raw_data):
    y = copy.deepcopy(raw_data['emotion'])
    data = raw_data.drop(columns = ['emotion'])
    x = []
    for i in range(data.shape[0]):
        sample = copy.deepcopy(data.loc[i,' pixels'])
        sample = sample.split(' ')
        sample = np.array(list(map(int, sample)))/255-1
        sample = sample.reshape((48,48,1))
        x.append(sample)
    x = np.array(x)
    y = y.to_frame()
    y = pd.get_dummies(y,columns = ['emotion'])
    return x,y
#generate sample images
def generate_image(pixels):
    plt.imshow(pixels)
    plt.show()
#generate training class weights used for fitting CNN and VGG
def get_class_weights(data):
    uniques = data.emotion.unique()
    dist_dict = {}
    for i in uniques:
        dist_dict[i] = data[data.emotion == i].shape[0]/data.shape[0]
    return dist_dict
#graph CNN and VGG model metrics
def metrics_grapher_CNN(history):
    length = int(len(history.history)/2)
    key_list = list(history.history.keys())
    for i in range(len(key_list)):
        if i==length:
            break
        plt.plot(history.history[key_list[i]])
        plt.plot(history.history[key_list[i+length]])
        plt.title('model '+key_list[i])
        plt.ylabel(key_list[i])
        plt.xlabel('epoch')
        plt.legend(['train', 'validation'], loc='upper left')
        plt.show()    
# print CNN and VGG model summary
def model_summary_CNN(model):
    model.summary()
    return None
#convert index labels into string for CNN and VGG
def convert_labels_CNN(ypred,ytrue,label_list):
    pred_indices = np.argmax(ypred,axis = 1)
    true_indices = np.argmax(ytrue,axis = 1)
    pred = []
    true = []
    for i in range(pred_indices.shape[0]):
        pred.append(label_list[pred_indices[i]])
        true.append(label_list[true_indices[i]])
    return pred,true
#convert index labels into string for other models
def convert_labels_general(ypred,ytrue,label_list):
    pred = []
    true = []
    for i in range(ypred.shape[0]):
        pred.append(label_list[ypred[i]])
        true.append(label_list[ytrue[i]])
    return pred,true
#draw confusion matrix
def draw_confusion(true,pred,label_list):
    cm = confusion_matrix(true,pred,labels = label_list)
    confusionMatrix = pd.DataFrame(cm,
                         index = label_list, 
                         columns = label_list)
    ax= plt.subplot()
    sns.heatmap(confusionMatrix, annot=True, fmt='g', ax=ax)
    ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels')
    ax.set_title('Confusion Matrix')
    return None
#display statistics for CNN and VGG model
def describe_CNN(model,history,xtest,ytest,label_list,is_vgg16):
    metrics_grapher_CNN(history)
    model_summary_CNN(model)
    if is_vgg16:
        xtest = np.repeat(xtest, repeats=3, axis=3)
    ypred_CNN = model.predict(xtest)
    CNNpred,true = convert_labels_CNN(ypred_CNN,ytest.values,label_list)
    print(classification_report(true, CNNpred))
    draw_confusion(true,CNNpred,label_list)
    return None
#display statistics for other models
def describe_general(model,xtest,ytest,label_list):
    ypred = model.predict(xtest)
    pred,true = convert_labels_general(ypred,ytest.to_numpy(),label_list)
    print(classification_report(true, pred))
    draw_confusion(true,pred,label_list)
    return accuracy_score(pred,true)
#describe data
def describe_data(training,validation,testing,xtrain,label_list):
    print('train size: '+str(training.shape[0]))
    print('validation size: '+str(validation.shape[0]))
    print('test size: '+str(testing.shape[0]))
    training_label = []
    valid_label = []
    testing_label = []
    for i in range(training.shape[0]):
        training_label.append(label_list[training['emotion'][i]])
    for i in range(testing.shape[0]):
        testing_label.append(label_list[testing['emotion'][i]])
    for i in range(validation.shape[0]):
        valid_label.append(label_list[validation['emotion'][i]])
    print('training set class distribution: ')
    plt.hist(np.array(training_label),bins = 7)
    plt.show()
    print('testing set class distribution: ')
    plt.hist(np.array(testing_label),bins = 7)
    plt.show()
    print('validation set class distribution: ')
    plt.hist(np.array(valid_label),bins = 7)
    plt.show()
    for i in label_list:
        print(i)
        generate_image(xtrain[training_label.index(i)])
        plt.show()
    return None
#definte VGG model
def VGG16_model(xtrain,ytrain,xval,yval,class_weights,filter_num,filter_size,pooling_size,dropout_rate):
    xtrain_colored = np.repeat(xtrain, repeats=3, axis=3)
    xval_colored = np.repeat(xval, repeats=3, axis=3)
    base_model = VGG16(weights="imagenet", include_top=False, input_shape= (48, 48, 3))
    base_model.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    model.compile( 'adam',loss='categorical_crossentropy',metrics=['accuracy'])
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3,verbose = 1)
    history = model.fit(xtrain_colored, ytrain,validation_data=(xval_colored, yval),class_weight = class_weights,epochs=100,
                            batch_size=100, callbacks=[callback])
    return model,history
#define CNN model
#conv - pooling - conv- pooling- flatten - dense - dense
def CNN_model(xtrain,ytrain,xval,yval,class_weights,filter_num,filter_size,pooling_size,dropout_rate):
    model = Sequential()
    model.add(Conv2D(filters = filter_num, kernel_size =  filter_size, activation='relu',padding="same" ,
                     input_shape=(48, 48, 1)))
    model.add(MaxPooling2D(pool_size=pooling_size))
    model.add(Dropout(dropout_rate))
    model.add(Conv2D(filters = filter_num, kernel_size = filter_size,padding="same" , activation='relu'))
    model.add(MaxPooling2D(pool_size=pooling_size))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    model.compile( 'adam',loss='categorical_crossentropy',metrics=['accuracy'])
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3,verbose = 1)
    history = model.fit(xtrain, ytrain,validation_data=(xval, yval),class_weight = class_weights,epochs=100,
                        batch_size=100, callbacks=[callback])
    return model,history